# Phase 5
     Model Building (Score 80 + 180 Extra credits):

 Each student must solve

    . Supervised Learning: Build a sentiment analysis model to predict positive and negative classes (Score 40)
    . Unsupervised Learning: Build a clustering model consisting of 2 clusters based on positive and negative reviews (Score 40)


In [1]:
# imports
import pandas as pd
import numpy as np
import pickle
import time

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import SGDClassifier

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:

#getting the data 
# importing pickled data for supervised and unsupervised 
sup_df=pd.read_pickle('pickledfiles/cl_fs_imdb_df.pkl') 
sup_df=sup_df[['feature_selected','label']] # cleaned and feature selected reviews 
#sup_df.shape 

unsup_df=pd.read_pickle('pickledfiles/unsup_df.pkl') # not cleaned or fs reviews 
#unsup_df.shape

as unsup_df is not cleaned would have to clean it 

In [3]:
def clinic_all_clean (mess):
    
    import string
    import re
    from nltk.corpus import stopwords
    from nltk import pos_tag
   
    from nltk.stem import WordNetLemmatizer
    from nltk.stem.porter import PorterStemmer 

    
    '''this function does the following:
        > it makes word lower case
        > it removes word containg numbers
        1. it removes words inside <> like <br>
        2. it removes punctuations 
        3. it removes stopwords 
        4. it stemmers
        5. it lemmatize
    '''
    
  
    
    mess = mess.lower()                 # make lowercase
    
    mess = re.sub(r'[^\x00-\x7F]+','', mess) # remove non ascii character 
    
    mess = re.sub('\(.*?\)', '', mess) # removes words like latin words (El Monte de las brujas)  that are inside ( )
    
    mess = re.sub('\<.*?\>', '', mess) # removes words like br that are inside < >
    
    mess = re.sub('\w*\d\w*', '', mess) # removes words containg  numbers
    
    nopunc = [x for x in mess if x not in string.punctuation] # remove all punc
     # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    stpwrd = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    # now it stemmers 
    stem = PorterStemmer()
    stem_w =[stem.stem(wrd) for wrd in stpwrd]
    
    
    # now its lemmatizes
    wnl = WordNetLemmatizer()
    lem_txt=[wnl.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else wnl.lemmatize(i) for i,j in pos_tag(stem_w)]
    
    return ' '.join(lem_txt) # removing join returns list of words aka bag of words 

In [4]:
def text_process(mess):
    import string

    return mess.split()

#### takes a while (where while ~ an hour or so. but its one time as i pickle it )

In [6]:
unsup_df['cleaned_review']=unsup_df.review.apply(clinic_all_clean)

unsup_df.to_pickle('pickledfiles/clean_unsup_df.pkl')

In [7]:
cl_unsup_df=pd.read_pickle('pickledfiles/clean_unsup_df.pkl')# cleaned unsup dataframe


In [8]:
cl_unsup_df.head()

,review,cleaned_review
0,"I admit, the great majority of films released ...",admit great major film releas say dozen major ...
1,"Take a low budget, inexperienced actors doubli...",take low budget inexperienc actor doubl produc...
2,"Everybody has seen 'Back To The Future,' right...",everybodi see back futur right whether like mo...
3,Doris Day was an icon of beauty in singing and...,dori day icon beauti sing act warm voic geniu ...
4,"After a series of silly, fun-loving movies, 19...",seri silli funlov movi big year dori day year ...


## Supervised Learning: Build a sentiment analysis model to predict positive and negative classes (Score 40)

In [9]:
# preparing data
x = sup_df.feature_selected
# encode the target strings
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y = le.fit_transform(sup_df.label)

In [10]:
from sklearn.model_selection import train_test_split

msg_train, msg_test, label_train, label_test = \
train_test_split(x, y, test_size=0.2)#80:20 pareto 

print(len(msg_train), len(msg_test), len(msg_train) + len(msg_test))

40000 10000 50000


# MultinomialNB
## creating pipeline

In [11]:
pipeline1 = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

start_time = time.time()
pipeline1.fit(msg_train,label_train)
cal_time1 = (time.time()- start_time)

predictions1 = pipeline1.predict(msg_test)
print(classification_report(predictions1,label_test),'time taken:',cal_time1)

             precision    recall  f1-score   support

          0       0.86      0.86      0.86      4996
          1       0.86      0.86      0.86      5004

avg / total       0.86      0.86      0.86     10000
 time taken: 3.118654727935791


In [12]:
# check the accuracy of the model
from sklearn.metrics import accuracy_score
accuracy1 = accuracy_score(label_test, predictions1)
accuracy1

0.8618

In [13]:
f1_score1=classification_report(predictions1,label_test).split( )[-2]
f1_score1

'0.86'

**Extra Credits:**
        
        . Supervised Learning: Compare the performance of different machine learning models, at least 2 (Score 40)

## LogisticRegression

In [14]:
pipeline5 = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier',  LogisticRegression()), 
])

In [15]:
start_time = time.time()

pipeline5.fit(msg_train,label_train)
cal_time5 = (time.time()- start_time)
predictions5 = pipeline5.predict(msg_test)

print(classification_report(predictions5,label_test))

             precision    recall  f1-score   support

          0       0.87      0.90      0.89      4786
          1       0.91      0.88      0.89      5214

avg / total       0.89      0.89      0.89     10000



In [16]:
# check the accuracy of the model
from sklearn.metrics import accuracy_score
accuracy5 = accuracy_score(label_test, predictions5)
accuracy5

0.889

In [17]:
f1_score5=classification_report(predictions5,label_test).split( )[-2]
f1_score5

'0.89'

## Stochastic Gradient Descent

In [18]:
pipeline9 = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier',  SGDClassifier()),  # train on TF-IDF vectors w/  DecisionTreeClassifier
])

In [19]:
start_time = time.time()

pipeline9.fit(msg_train,label_train)
cal_time9 = (time.time()- start_time)
predictions9 = pipeline9.predict(msg_test)

print(classification_report(predictions9,label_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


             precision    recall  f1-score   support

          0       0.87      0.90      0.88      4792
          1       0.91      0.87      0.89      5208

avg / total       0.89      0.89      0.89     10000



In [20]:
# check the accuracy of the model
from sklearn.metrics import accuracy_score
accuracy9 = accuracy_score(label_test, predictions9)
accuracy9

0.8864

In [21]:
f1_score9=classification_report(predictions9,label_test).split( )[-2]
f1_score9

'0.89'

### comparing the performance

In [22]:
alist = [('naive bayes',accuracy1,f1_score1,cal_time1),
          
          ('Logistic Regression',accuracy5,f1_score5,cal_time5),
          
          ('Stochastic Gradient Descent',accuracy9,f1_score9,cal_time9),
          ]
    
pd.DataFrame(np.array(alist),columns=['model','accuracy','f1_score','time'])

,model,accuracy,f1_score,time
0,naive bayes,0.8618,0.86,3.118654727935791
1,Logistic Regression,0.889,0.89,3.0687906742095947
2,Stochastic Gradient Descent,0.8864,0.89,3.5285329818725586


--------------------

# Unsupervised Learning:
Build a clustering model consisting of 2 clusters based on positive and negative reviews (Score 40)

## KMeans Clustering

In [23]:
def phase2_cleaning(mess):
    import string 
    added_stopwords = ['film', 'movi', 'one', 'see', 'make', 'like', 'get', 'show', 'charact']
        # remove addition stopwords that we had created in Part2 
    stpwrd = [word for word in mess.split() if word.lower() not in added_stopwords]
    return stpwrd

In [24]:
import pandas as pd
cl_unsup_df=pd.read_pickle('pickledfiles/clean_unsup_df.pkl')# cleaned unsup dataframe
msg_train=cl_unsup_df.cleaned_review

In [25]:

from sklearn.feature_extraction.text import TfidfVectorizer

#vectorizer
vectorizer=TfidfVectorizer(analyzer=phase2_cleaning)
X=vectorizer.fit_transform(msg_train)
true_k = 2

In [26]:
#model
km=KMeans(n_clusters=true_k,n_init=3,max_iter=100)

km.fit(X)

#looking at clusters 

centroid=km.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    print('cluster id : %d:' %i),
    for j in centroid[i,:10]:
        print('%s'% terms[j])

cluster id : 0:
stori
great
time
love
good
play
well
scene
also
watch
cluster id : 1:
watch
bad
good
realli
think
time
would
even
dont
go


In [27]:
print("Prediction")

Y = vectorizer.transform(['bad movie dont watch'])
prediction = km.predict(Y)
print(prediction)

Y = vectorizer.transform(['great movie loved it '])
prediction = km.predict(Y)
print(prediction)

Prediction
[1]
[0]


## Hierarchical clustering¶

In [28]:
#data subset
lk=msg_train[:1000]
#vectorizer
cv = CountVectorizer()
jk=cv.fit(lk)
zz=jk.transform(lk)

#sparse to dense
ss=zz.toarray()

#model
sg=AgglomerativeClustering()
sg.fit(ss)
#


AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward', memory=None, n_clusters=2,
            pooling_func=<function mean at 0x000001EEDA517BF8>)

**Divide the data into 4 clusters to enable finding more classes. Analyse each cluster and try to find the correct label for the new cluster. Repeat clustering until 4 new labels can be found, other than the original labels (positive and negative) (Score 50)**

In [29]:

from sklearn.feature_extraction.text import TfidfVectorizer

#vectorizer
vectorizer=TfidfVectorizer(analyzer=phase2_cleaning)
X=vectorizer.fit_transform(msg_train)
true_k = 4

In [30]:
#model
km=KMeans(n_clusters=true_k,n_init=3,max_iter=100)

km.fit(X)

#looking at clusters 

centroid=km.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    print('cluster id : %d:' %i),
    for j in centroid[i,:10]:
        print('%s'% terms[j])

cluster id : 0:
great
stori
time
good
play
love
scene
well
also
watch
cluster id : 1:
funni
comedi
laugh
watch
joke
good
great
time
realli
think
cluster id : 2:
realli
think
watch
good
stori
would
go
time
peopl
end
cluster id : 3:
bad
act
watch
worst
even
ever
good
wast
dont
time


###  now that we divided data into 4 clusters lets analyze cluster and decide classes

i could go through manually but there wont be anything tangible to backup my claim so trying something crazy


In [31]:
from textblob import TextBlob
from nltk.corpus import stopwords
import string
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from sklearn import cluster
import re

def create_polarity_vector(all_adjectives):
    # NLTK Vader sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    #Taking compound score as one feature, (compund score = -ve indicates negative sentiments)
    #(compund score = +ve indicates positive sentiments, ie compound value is normalization b/w +ve and -ve)
    feature_vector=[1 if sid.polarity_scores(i)['compound']>=0 else -1 for i in all_adjectives]
    
    return feature_vector



# utility function to extract all adjectives from text document
def extract_adjectives(text):
    tokenized_text= nltk.word_tokenize(text) # tokenizes sentences to words 
    tagged_text = nltk.pos_tag(tokenized_text) # creates tag for each word 
    all_adjectives = list({i[0] for i in tagged_text if i[1] == 'JJ'}) #JJ: adjective or numeral, ordinal
    return all_adjectives

# Cleaning up text  # does not lemma and steming as it may hinder getting proper tag 
def clean_text(text):    
    
    text = text.lower()                 # make lowercase
    
    text = re.sub('\<.*?\>', '', text) # removes words like br that are inside < >
    
    text = re.sub('\w*\d\w*', '', text) # removes words containg  numbers
    
    #stopword removal
    stop_word=list(stopwords.words('english'))
    toke=list(text.split(' '))
    text = ' '.join([i for i in toke if i not in stop_word])
    
    #punctutaion removal
    t_lator=str.maketrans('','',string.punctuation)
    text=text.translate(t_lator)
    remove_digits = str.maketrans('', '', string.digits)
    text = text.translate(remove_digits)    
    
    #removing special symbol
    for i in '“”—':
        text = text.replace(i, ' ')
        
    return text


C:\ProgramData\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [32]:
centroid[3].size

166391

In [33]:
clu1 = [] # we can play with amount of words we need from each cluster however selecting very high number diminishes the difference

for j in centroid[1,:800]:
        #print('%s'% terms[j])
        clu1.append(terms[j])
        
        
clu2 = []
for j in centroid[2,:800]:
        #print('%s'% terms[j])
        clu2.append(terms[j])
        

clu3 = []
for j in centroid[3,:800]:
        #print('%s'% terms[j])
        clu3.append(terms[j])
        
clu0 = []
for j in centroid[0,:800]:
        #print('%s'% terms[j])
        clu0.append(terms[j])

In [34]:
adj_0=extract_adjectives(' '.join(clu0))
adj_1=extract_adjectives(' '.join(clu1))
adj_2=extract_adjectives(' '.join(clu2))
adj_3=extract_adjectives(' '.join(clu3))


In [35]:
def create_polarity_vector(all_adjectives):
    # NLTK Vader sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    #Taking compound score as one feature, (compund score = -ve indicates negative sentiments)
    #(compund score = +ve indicates positive sentiments, ie compound value is normalization b/w +ve and -ve)
    feature_vector=[1 if sid.polarity_scores(i)['compound']>=0 else -1 for i in all_adjectives]
    
    return feature_vector


In [36]:
sid = SentimentIntensityAnalyzer()

In [37]:
sid.polarity_scores(' '.join(adj_0))# ++ positive

{'neg': 0.175, 'neu': 0.552, 'pos': 0.272, 'compound': 0.9607}

In [38]:
sid.polarity_scores(' '.join(adj_1))# + positive

{'neg': 0.211, 'neu': 0.532, 'pos': 0.257, 'compound': 0.8591}

In [39]:
sid.polarity_scores(' '.join(adj_2))# - negative

{'neg': 0.236, 'neu': 0.518, 'pos': 0.246, 'compound': -0.3561}

In [40]:
sid.polarity_scores(' '.join(adj_3)) # -- negative

{'neg': 0.204, 'neu': 0.616, 'pos': 0.18, 'compound': -0.765}

#conclusion 
.note: each time you run code the cluster name or id may interchange however values remain somewhat same 

>**postive:**
    
    .cluster 0 is most positive ++
    .cluster 1 is positive +

>**negative:**
    
    .cluster 2 is most negative --
    .cluster 3 is negative -
    
    

#positive and negative are the main clusters 
>  .clusters (3,0) are sub-cluster of positive 

>   .clusters(2,1) are sub_ cluster of negative 

### Active Learning: Cluster the training dataset and try and find the genre. Manually annotate the cluster and then try to find the labels in the new testing dataset. (Score 50)

# performed it in new notebook i.e Part5(active)